In [1]:
# install libraries
# !pip install hazm
# !pip install pandas
# !pip install gensim

In [2]:
# libraries
import codecs
import os
import json
import tqdm
from collections import Counter
from nltk import FreqDist
import pandas as pd
import numpy as np
import itertools
from hazm import *
from gensim.models.fasttext import FastText

In [3]:
import json

DIRNAME = '../dataset'
data = []
for i in range(1, 8):
    with open(f'{DIRNAME}/namnak-p{i}.json', 'r', encoding="utf-8") as f:
        data.extend(json.loads(f.read()))
    with open(f'{DIRNAME}/hidoctor-p{i}.json', 'r', encoding="utf-8") as f:
        data.extend(json.loads(f.read()))


In [4]:
len(data)

4322

### normalization

In [5]:
# normalization
from hazm import *

normalizer = Normalizer()

normalized_data = []
for item in tqdm.tqdm(data):
    normalized_data.append({"title" : normalizer.normalize(item['title']), "text":normalizer.normalize(item['text']), 'link':item['link']})

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:06<00:00, 656.08it/s]


In [6]:
import random
print(random.sample(normalized_data, 10)[0]['text'][:1000])

«سو ماکسلی» مجری تلویزیونی در سن ۵۵ سالگی همجنان جوان و زیباست. این چهره‌ی مشهور در یک مصاحبه راز جوانی و زیبایی خود را بسیار ساده بیان کرد. رازهای زیبایی و جوانی مجری مشهور ۵۵ ساله تلویزیون: در دنیا کم نیستند افرادی که در دهه ۵۰ و یا ۶۰ زندگی شان همچنان زیبایی و جوانی در چهره شان دیده می‌شود، سلبریتی‌ها و چهره‌های مشهور در میان این دسته افراد بیشتر دیده می‌شود و دانستن رازهای زیبایی آنها برای بسیاری از افراد جالب و کاربردی است. یکی از این چهره‌ها خانم «سو مالکسی» ۵۵ ساله است که سالهاست کار اجرای تلویزیونی انجام می‌دهد. دیلی میل در مصاحبه‌ای با او دلایل و شگردهای جوان ماندنش را منتشر کرد. روش‌های ساده‌ای که راز جوانی و زیبایی سو ماکسلی هستند چهره مشهور تلویزیونی در این گزارش رازهای جوانی و زیبایی اش را بسیار ساده خواند و در این مورد گفت: برای آنکه مدت طولانی‌تری جوان و زیبا بمانید: موهایتان را کش نیاورید! من باور ندارم که موهای کوتاه بهتر است اما به این هم معتقد نیستم که تلاش برای داشتن موهایی بلند و صاف ارزش انجام عادات نابود کننده موها را داشته باشد چرا که اکثر خانم‌ها با داشتن موهای

### stopwords

In [7]:
# Persian Stopwords
# https://github.com/sobhe/hazm/blob/master/hazm/data/stopwords.dat
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

In [8]:
def create_dataframe_freq(data, stopwords=[]):
    tokens = []
    for item in tqdm.tqdm(data):
        tokens.extend([_ for _ in word_tokenize(item['text'])  if _ not in stopwords])
    return pd.DataFrame(FreqDist(tokens).most_common(30)), tokens

In [9]:
freq_analysis, tokens = create_dataframe_freq(normalized_data, stopwords)

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:13<00:00, 322.08it/s]


In [10]:
freq_analysis

,0,1
0,.,135766
1,،,98207
2,:,18986
3,استفاده,15644
4,مصرف,13151
5,بدن,12752
6,کاهش,9786
7,درمان,9637
8,پوست,8233
9,آب,8015


In [11]:
custom_stop_words = [normalizer.normalize(x.strip()) for x in codecs.open('custom_stopwords.txt','r','utf-8').readlines()]

In [12]:
total_stop_words = custom_stop_words + stopwords

In [13]:
freq_analysis, tokens = create_dataframe_freq(normalized_data, total_stop_words)

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:12<00:00, 340.19it/s]


In [14]:
freq_analysis

,0,1
0,استفاده,15644
1,مصرف,13151
2,بدن,12752
3,کاهش,9786
4,درمان,9637
5,پوست,8233
6,آب,8015
7,کمک,7871
8,بیماری,7659
9,صورت,7647


In [15]:
tokens = []
for item in tqdm.tqdm(normalized_data):
    tokens.append([_ for _ in word_tokenize(item['text'])  if _ not in total_stop_words])

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:12<00:00, 346.26it/s]


## fasttext

In [16]:
ft_model = FastText(
    sg=1, # use skip-gram: usually gives better results
    vector_size=110, # embedding dimension (default)
    window=10, # window size: 10 tokens before and 10 tokens after to get wider context
    min_count=8, # only consider tokens with at least n occurrences in the corpus
    negative=15, # negative subsampling: bigger than default to sample negative examples more
    epochs=10,
    sentences=tokens,
    min_n=2, # min character n-gram
    max_n=8 # max character n-gram
)

os.makedirs('../models', exist_ok=True)
ft_model.save('../models/_fasttext.model') #save

In [17]:
ft_model = FastText.load('../models/_fasttext.model') #load

In [18]:
ft_model.wv.most_similar('کرونا')

[('ویروس', 0.8722943663597107),
 ('کروناست', 0.8381739258766174),
 ('کرونای', 0.8063480854034424),
 ('کروناویروس', 0.8027466535568237),
 ('کووید', 0.7813413739204407),
 ('فلورونا', 0.772040605545044),
 ('کرونایی', 0.7522709965705872),
 ('کووید-', 0.7389469146728516),
 ('واکسیناسیون', 0.7212578058242798),
 ('امیکرون', 0.7207950949668884)]

In [19]:
ft_model.wv['کرونا']

array([-0.7250474 ,  0.07823099,  0.29230082,  0.68657804, -0.61483973,
        0.30243918,  0.06679444, -0.3924599 , -0.05695422,  0.05899319,
       -0.14479662, -0.12202868,  0.1882111 ,  0.2816578 , -0.3259505 ,
        0.6796811 , -0.2772728 , -0.09960154, -0.3220463 , -0.47524375,
       -0.36247408,  0.76155543,  0.63111275, -0.04677137, -0.47269276,
       -0.3001579 , -0.3537347 ,  0.22579752, -0.64531213, -0.25180787,
        0.33996728, -0.03389461,  0.5828695 ,  0.01973744,  0.48357362,
       -0.2134468 ,  0.3698861 ,  0.34546447, -0.07447707, -0.0044238 ,
       -0.37520525, -0.580367  , -0.39302385,  0.1268842 ,  0.2030868 ,
        0.04934395,  0.54586035,  0.00513826,  0.76262003, -0.31198397,
       -0.3645133 ,  0.2940068 , -0.07128896, -0.13759333,  0.37387913,
       -0.05040118, -0.12391483,  0.27047083, -0.07894358,  0.219805  ,
        0.46882686,  0.24460569,  0.0400802 ,  0.3320905 , -0.35880113,
        0.10950287,  0.18177062, -0.4465768 ,  0.25088903, -0.07

In [20]:
docs_embs = []
for item in tqdm.tqdm(normalized_data):
    doc_tokens = [_ for _ in word_tokenize(item['text'])  if _ not in total_stop_words]
    emb = np.zeros(ft_model.wv.vector_size)
    for token in doc_tokens:
        emb += ft_model.wv[token]
    emb /= len(doc_tokens)
        
    emb2 = np.zeros(ft_model.wv.vector_size)
    title_tokens = [_ for _ in word_tokenize(item['title'])  if _ not in total_stop_words]
    for token in title_tokens:
        emb2 += ft_model.wv[token]
    emb2 /= len(title_tokens)
    emb = ((emb * 9) + (emb2)) / 10
    
    docs_embs.append({'title':item['title'], 'emb': emb.tolist(), 'link':item['link']})

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:29<00:00, 148.27it/s]


In [21]:
docs_embs[0]

{'title': 'ردپای مواد مخدر روی مغز نوجوانان!!',
 'emb': [0.10109286424673725,
  0.007775328069034551,
  -0.009126353548146391,
  0.19213834481118625,
  -0.17877776709563237,
  0.24183827214124207,
  0.02962466357656352,
  -0.14365105153853644,
  0.040164351614481786,
  -0.054183018975273056,
  -0.08472046917464895,
  -0.09344916679771367,
  0.21201917181244756,
  0.03219202781113534,
  -0.10971008255905783,
  -2.0643286950097763e-05,
  -0.16413231227372432,
  0.12032576304732767,
  0.06270471627334041,
  0.0140885817403305,
  0.08782649893626937,
  -0.0828377169019815,
  -0.07380846872173291,
  -0.007343844630022293,
  -0.2413689334524153,
  0.11860690090664153,
  0.1016757672833972,
  0.06049907918770706,
  -0.1089268443877259,
  -0.136351546651815,
  0.23186043359909494,
  0.0032793766662882083,
  -0.07030410317912271,
  0.013307222273997638,
  0.19363452810549694,
  -0.18895299550806474,
  -0.12901752265219665,
  -0.0416637810611262,
  0.05472558388168898,
  0.24247161098993925,
  0

In [22]:
# save json
with open(f'../models/fasttext_docs_embedding.json', 'w', encoding="utf-8") as f:
    json.dump(docs_embs, f)

In [23]:
# load json
with open(f'../models/fasttext_docs_embedding.json', 'r', encoding="utf-8") as f:
    docs_embs = json.loads(f.read())

In [24]:
docs_embs[0]

{'title': 'ردپای مواد مخدر روی مغز نوجوانان!!',
 'emb': [0.10109286424673725,
  0.007775328069034551,
  -0.009126353548146391,
  0.19213834481118625,
  -0.17877776709563237,
  0.24183827214124207,
  0.02962466357656352,
  -0.14365105153853644,
  0.040164351614481786,
  -0.054183018975273056,
  -0.08472046917464895,
  -0.09344916679771367,
  0.21201917181244756,
  0.03219202781113534,
  -0.10971008255905783,
  -2.0643286950097763e-05,
  -0.16413231227372432,
  0.12032576304732767,
  0.06270471627334041,
  0.0140885817403305,
  0.08782649893626937,
  -0.0828377169019815,
  -0.07380846872173291,
  -0.007343844630022293,
  -0.2413689334524153,
  0.11860690090664153,
  0.1016757672833972,
  0.06049907918770706,
  -0.1089268443877259,
  -0.136351546651815,
  0.23186043359909494,
  0.0032793766662882083,
  -0.07030410317912271,
  0.013307222273997638,
  0.19363452810549694,
  -0.18895299550806474,
  -0.12901752265219665,
  -0.0416637810611262,
  0.05472558388168898,
  0.24247161098993925,
  0

In [25]:
class FastTextEmb:
    def __init__(self):
        self.ft_model = FastText.load('../models/_fasttext.model')
        with open(f'../models/fasttext_docs_embedding.json', 'r', encoding="utf-8") as f:
            self.docs_embs = json.loads(f.read())
        self.normalizer = Normalizer()
        stopwords = [self.normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]
        custom_stop_words = [self.normalizer.normalize(x.strip()) for x in codecs.open('custom_stopwords.txt','r','utf-8').readlines()]
        self.total_stop_words = custom_stop_words + stopwords
    
    def print_similars(self, query, k=10):
        ls = self.get_query(query, k)
        for i, item in enumerate(ls):
            print(f'{i + 1}- title: {item[0]}')
            print(f'{i + 1}- link: {item[1]}')
            print('-------------------------')
    
    def get_query(self, query, k=10):
        query_tokens = [_ for _ in word_tokenize(self.normalizer.normalize(query)) if _ not in self.total_stop_words]
        emb = np.zeros(self.ft_model.wv.vector_size)
        for token in query_tokens:
            emb += self.ft_model.wv[token]
        emb /= len(query_tokens)
        return self.nearest_neighbor(emb, self.docs_embs, k)
    
    def cosine_similarity(self, vector_1: np.ndarray, vector_2: np.ndarray) -> float:
        return np.dot(vector_1, vector_2)/(np.linalg.norm(vector_1) *
                                          np.linalg.norm(vector_2))
    
    def nearest_neighbor(self, v, doc_embs, k):
        data = {}
        for doc in doc_embs:
            data[doc['title']] = (self.cosine_similarity(v, doc['emb']), doc['link'])
        return [(k,v[1]) for k, v in sorted(data.items(), key=lambda item: item[1][0])][::-1][:k]

In [26]:
FastTextEmb().print_similars('ویروس کرونا')

1- title: مراقب افراد بدون علامت باشید / افراد ناقل تا ۶۰درصد قدرت شیوع دارند
1- link: https://www.hidoctor.ir/352514_%d9%85%d8%b1%d8%a7%d9%82%d8%a8-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d8%a8%d8%af%d9%88%d9%86-%d8%b9%d9%84%d8%a7%d9%85%d8%aa-%d8%a8%d8%a7%d8%b4%db%8c%d8%af-%d8%a7%d9%81%d8%b1%d8%a7%d8%af-%d9%86%d8%a7%d9%82.html/
-------------------------
2- title: نوع جدید ویروس کرونا در ویتنام
2- link: https://www.hidoctor.ir/355062_%d9%86%d9%88%d8%b9-%d8%ac%d8%af%db%8c%d8%af-%d9%88%db%8c%d8%b1%d9%88%d8%b3-%da%a9%d8%b1%d9%88%d9%86%d8%a7-%d8%af%d8%b1-%d9%88%db%8c%d8%aa%d9%86%d8%a7%d9%85.html/
-------------------------
3- title: کرونا دلتا دقیقا چیست و چرا خطرناک‌تر است؟
3- link: https://namnak.com/what-is-delta-covid.p82282
-------------------------
4- title: بهترین و تنها راه پیشگیری از ابتلا به کرونا دلتا
4- link: https://namnak.com/covid-19-delta-variant.p82669
-------------------------
5- title: بهترین راه برای کرونا نگرفتن بعد از واکسن
5- link: https://namnak.com/covid-after-vaccination.p

In [27]:
FastTextEmb().print_similars('دندان درد')

1- title: پر کردن دندان بدون نیاز به دندانپزشک
1- link: https://namnak.com/پر-کردن-دندان.p29166
-------------------------
2- title: هر چیزی که باید درباره دندان عقل بدانید
2- link: https://www.hidoctor.ir/357156_%d9%87%d8%b1-%da%86%db%8c%d8%b2%db%8c-%da%a9%d9%87-%d8%a8%d8%a7%db%8c%d8%af-%d8%af%d8%b1%d8%a8%d8%a7%d8%b1%d9%87-%d8%af%d9%86%d8%af%d8%a7%d9%86-%d8%b9%d9%82%d9%84-%d8%a8%d8%af%d8%a7%d9%86%db%8c%d8%af.html/
-------------------------
3- title: دندان عقل و کشیدن آن خوب است یا بد؟
3- link: https://namnak.com/کشیدن-دندان-عقل.p39551
-------------------------
4- title: درمان آسان دندان‌های حساس
4- link: https://www.hidoctor.ir/356749_%d8%af%d8%b1%d9%85%d8%a7%d9%86-%d8%a2%d8%b3%d8%a7%d9%86-%d8%af%d9%86%d8%af%d8%a7%d9%86-%d9%87%d8%a7%db%8c-%d8%ad%d8%b3%d8%a7%d8%b3.html/
-------------------------
5- title: علائم حساسیت عاج دندان و درمان آن
5- link: https://namnak.com/حساسیت-عاج-دندان.p41805
-------------------------
6- title: عصب کشی دندان دقیقا چگونه انجام می‌شود؟
6- link: https://namna

In [28]:
FastTextEmb().print_similars('ریزش مو')

1- title: چهار تا از بهترین شامپو‌ها برای جلوگیری از ریزش مو
1- link: https://www.hidoctor.ir/354908_%da%86%d9%87%d8%a7%d8%b1-%d8%aa%d8%a7-%d8%a7%d8%b2-%d8%a8%d9%87%d8%aa%d8%b1%db%8c%d9%86-%d8%b4%d8%a7%d9%85%d9%be%d9%88-%d9%87%d8%a7-%d8%a8%d8%b1%d8%a7%db%8c-%d8%ac%d9%84%d9%88%da%af%db%8c%d8%b1%db%8c.html/
-------------------------
2- title: انواع بانک مو در کاشت مو
2- link: https://www.hidoctor.ir/354599_%d8%a7%d9%86%d9%88%d8%a7%d8%b9-%d8%a8%d8%a7%d9%86%da%a9-%d9%85%d9%88-%d8%af%d8%b1-%da%a9%d8%a7%d8%b4%d8%aa-%d9%85%d9%88.html/
-------------------------
3- title: مو کاشتم ولی داره میریزه، چرا؟
3- link: https://namnak.com/losing-hair-after-transplant.p78577
-------------------------
4- title: آیا کاشت مو می‌تواند ریزش ژنتیکی مو را درمان کند؟
4- link: https://www.hidoctor.ir/353142_%d8%a2%db%8c%d8%a7-%da%a9%d8%a7%d8%b4%d8%aa-%d9%85%d9%88-%d9%85%db%8c-%d8%aa%d9%88%d8%a7%d9%86%d8%af-%d8%b1%db%8c%d8%b2%d8%b4-%da%98%d9%86%d8%aa%db%8c%da%a9%db%8c-%d9%85%d9%88-%d8%b1%d8%a7-%d8%af.html/
-------

In [29]:
FastTextEmb().print_similars('تنبلی چشم', k=5)

1- title: علت خشکی و قرمزی چشم و درمان آن
1- link: https://namnak.com/قرمزی-چشم.p49476
-------------------------
2- title: علت ضعیف شدن چشم و درمان آن
2- link: https://namnak.com/ضعیف-شدن-چشم.p39635
-------------------------
3- title: چگونه می‌توان از شر قرمزی چشم خلاص شد؟
3- link: https://www.hidoctor.ir/354578_%da%86%da%af%d9%88%d9%86%d9%87-%d9%85%db%8c-%d8%aa%d9%88%d8%a7%d9%86-%d8%a7%d8%b2-%d8%b4%d8%b1-%d9%82%d8%b1%d9%85%d8%b2%db%8c-%da%86%d8%b4%d9%85-%d8%ae%d9%84%d8%a7%d8%b5-%d8%b4%d8%af%d8%9f.html/
-------------------------
4- title: دلیل اصلی سنگینی پلک چشم و بهترین درمان خانگی آن
4- link: https://namnak.com/heavy-eyelids-treatment.p80060
-------------------------
5- title: علت پف صبحگاهی چشم‌ها بعد بیدار شدن از خواب + درمان
5- link: https://namnak.com/علت-پف-کردن-چشم-ها.p47447
-------------------------
